In [1]:
## database imports
from database.market import Market
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## modeler import
from modeler_strats.universal_modeler import UniversalModeler

## strategy impors
from strategy.speculation import Speculation as spec
from strategy.classification import Classification as classer

## miscellaneous
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
from statistics import variance
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
current_db = ADatabase("current")
umod = UniversalModeler()

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [5]:
classification_factors = []
classification_factors.append("d1")
classification_factors.append("d2")
classification_factors.append("d3")
classification_factors.append("rolling14")

In [6]:
factors = [str(x) for x in range(14)]

In [7]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose"
]
included_columns.extend(factors)
included_columns.extend(classification_factors)

In [8]:
strats = ["spec","rolling","window"]

In [9]:
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data = spec.transform(ticker_data)
        ticker_data = classer.transform(ticker_data)
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
data = pd.concat(training_sets)
data = data.dropna()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:42<00:00, 11.44it/s]


In [10]:
market.connect()
sp500 = market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
market.disconnect()

In [11]:
prediction_slice = data[data["year"]==start_date.year].reset_index(drop=True)

In [12]:
speculation_db.connect()
models = speculation_db.retrieve("models")
speculation_db.disconnect()

In [13]:
classification_db.connect()
classification_models = classification_db.retrieve("models")
classification_db.disconnect()

In [14]:
simulation = umod.recommend(models,prediction_slice.copy(),factors)
simulation["week"] = simulation["week"] + 1
simulation["prediction"] = (simulation["cat_prediction"] + simulation["xgb_prediction"] + simulation["tf_prediction"]) / 3
speculation_db.connect()
speculation_db.drop("predictions")
speculation_db.store("predictions",simulation)
speculation_db.disconnect()

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-03 11:17:12         2522
metadata.json                                  2023-03-03 11:17:12           64
variables.h5                                   2023-03-03 11:17:12        84160
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...metrics\mean
......vars
.........0
.........1
...metrics\mean_absolute_percentage_error
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
321/321 [==============================] - 0s 553us/step


In [16]:
classification_simulation = umod.recommend(classification_models,prediction_slice.copy(),classification_factors)
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation["classification_prediction"] = ((classification_simulation["cat_prediction"] + \
                                                          classification_simulation["xgb_prediction"] + \
                                                          classification_simulation["tf_prediction"]) / 3) > 0.5
classification_simulation["classification_prediction"] = [int(x) for x in classification_simulation["classification_prediction"]]
classification_db.connect()
classification_db.drop("predictions")
classification_db.store("predictions",classification_simulation)
classification_db.disconnect()

TypeError: a bytes-like object is required, not 'XGBClassifier'